In [1]:
import json

In [5]:
# Let's see what fields are present in the card data, so we can accurately model the data and load it into a database.
with open('data/cards.json') as f:
    data = json.load(f)

# Keys are card sets, which also appear in individual card data in the `cardSet` field.
data.keys()

dict_keys(['Reward', 'Credits', 'System', 'Tavern Brawl', 'Missions', 'The Grand Tournament', 'Basic', 'Hero Skins', 'Karazhan', 'The League of Explorers', 'Blackrock Mountain', 'Debug', 'Promo', 'Mean Streets of Gadgetzan', 'Naxxramas', 'Goblins vs Gnomes', 'Classic', 'Whispers of the Old Gods'])